In [176]:
import pandas as pd
import matplotlib.pyplot as plt
import calendar
from datetime import datetime 


In [177]:
#functions
#get weekday
def weekday(date):  
    return calendar.day_name[date.weekday()] 

#Adding days to date
def add_days_to_date(date,days):
    return date + datetime.timedelta(days=days)

In [178]:
df = pd.read_csv("5 - Muesli Project raw data - Orders.csv", header = 1)
df2 = pd.read_csv("5 - DF2 Muesli Project raw data - Order Process Data.csv")
warehouse_workdays = ["Monday", "Tuesday","Wednesday","Thursday","Friday"]
transport_workdays = ["Monday","Wednesday","Friday"]
warehousestorage_time = 1
warehouse_prep_time = 2
express = warehousestorage_time
no_express = warehousestorage_time + warehouse_prep_time

In [179]:
#adjust column names
df.columns = df.columns.str.replace(" ","_")
df.columns = map(str.lower, df.columns)

In [180]:
df2.columns = df2.columns.str.replace(" ","_")
df2.columns = map(str.lower, df2.columns)

In [181]:
#merging both df
df = df.drop('ship_mode', axis=1)
df2 = df2[['order_id', 'ship_mode']]
df = df.merge(df2, on='order_id', how='left')

df.head()

,index,order_id,order_date,customer_id,customer_name,origin_channel,country/region,city,state,postal_code,region,category,sub-category,product_id,sales,quantity,discount,profit,ship_mode
0,27,CA-2019-121755,16/1/2019,EH-13945,Eric Hoffmann,Email,United States,Los Angeles,California,90049.0,West,Special Projects Muesil,Gluten Free,TEC-AC-10003027,90.57,3,0.0,11.7741,Standard Processing
1,27,CA-2019-121755,16/1/2019,EH-13945,Eric Hoffmann,Email,United States,Los Angeles,California,90049.0,West,Special Projects Muesil,Gluten Free,TEC-AC-10003027,90.57,3,0.0,11.7741,Standard Processing
2,45,CA-2019-118255,11/3/2019,ON-18715,Odella Nelson,Sales,United States,Eagan,Minnesota,55122.0,Central,Special Projects Muesil,Gluten Free,TEC-AC-10000171,45.98,2,0.0,19.7714,Express
3,45,CA-2019-118255,11/3/2019,ON-18715,Odella Nelson,Sales,United States,Eagan,Minnesota,55122.0,Central,Special Projects Muesil,Gluten Free,TEC-AC-10000171,45.98,2,0.0,19.7714,Express
4,48,CA-2019-169194,20/6/2019,LH-16900,Lena Hernandez,Email,United States,Dover,Delaware,19901.0,East,Special Projects Muesil,Gluten Free,TEC-AC-10002167,45.00,3,0.0,4.9500,Standard Processing


In [182]:
#order_date into date
df["order_date"] = df["order_date"].apply(lambda date: datetime.strptime(str(date), "%d/%m/%Y"))

In [183]:
df.shape

(21620, 19)

In [184]:
df.columns


Index(['index', 'order_id', 'order_date', 'customer_id', 'customer_name',
       'origin_channel', 'country/region', 'city', 'state', 'postal_code',
       'region', 'category', 'sub-category', 'product_id', 'sales', 'quantity',
       'discount', 'profit', 'ship_mode'],
      dtype='object')

In [185]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21620 entries, 0 to 21619
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   index           21620 non-null  int64         
 1   order_id        21620 non-null  object        
 2   order_date      21620 non-null  datetime64[ns]
 3   customer_id     21620 non-null  object        
 4   customer_name   21620 non-null  object        
 5   origin_channel  21620 non-null  object        
 6   country/region  21620 non-null  object        
 7   city            21620 non-null  object        
 8   state           21620 non-null  object        
 9   postal_code     21591 non-null  float64       
 10  region          21620 non-null  object        
 11  category        21620 non-null  object        
 12  sub-category    21620 non-null  object        
 13  product_id      21620 non-null  object        
 14  sales           21620 non-null  float64       
 15  qu

In [186]:
#create column: Weekdays of order_date
df["order_date_weekday"] = df["order_date"].apply(lambda date: weekday(date))

In [187]:
#check
df.order_date_weekday.unique()

array(['Wednesday', 'Monday', 'Thursday', 'Saturday', 'Tuesday', 'Friday',
       'Sunday'], dtype=object)

In [188]:
#delivery day: adding days whether express or not
import datetime
df["order_express_considered"] = df.apply(lambda x: add_days_to_date(x["order_date"],express) if x["ship_mode"] == "Express" else add_days_to_date(x["order_date"],no_express), axis = 1 )

In [189]:
#delivery day: adding days whether order date is on warehouse workdays
df["order_leaving_warehouse"] = df.apply(lambda x: add_days_to_date(x["order_express_considered"],days =2) if x["order_date_weekday"] == "Saturday" else add_days_to_date(x["order_express_considered"],days =1) if x["order_date_weekday"] == "Sunday" else x["order_express_considered"], axis = 1)

In [190]:
#adding new weekday column for "order_leaving_warehouse"
df["order_leaving_warehouse_weekday"] = df["order_leaving_warehouse"].apply(lambda date: weekday(date))

In [191]:
#delivery day: adding days considering logistics
df["delivery_day"] = df.apply(lambda x: add_days_to_date(x["order_leaving_warehouse"],days =5) if x["order_leaving_warehouse_weekday"] == "Wednesday" else add_days_to_date(x["order_leaving_warehouse"],days =3) if x["order_leaving_warehouse_weekday"] == "Monday" else add_days_to_date(x["order_leaving_warehouse"],days =6) if x["order_leaving_warehouse_weekday"] == "Tuesday" else add_days_to_date(x["order_leaving_warehouse"],days =4) if x["order_leaving_warehouse_weekday"] == "Thursday" else add_days_to_date(x["order_leaving_warehouse"],days =3) if x["order_leaving_warehouse_weekday"] == "Friday" else add_days_to_date(x["order_leaving_warehouse"],days =5) if x["order_leaving_warehouse_weekday"] == "Saturday" else add_days_to_date(x["order_leaving_warehouse"],days =4) if x["order_leaving_warehouse_weekday"] == "Sunday" else "", axis = 1)

In [192]:
#delivery_time calculation
df["delivery_time[days]"] = df.apply(lambda x : x["delivery_day"] - x["order_date"], axis = 1)

In [193]:
#check
df.ship_mode.unique()

array(['Standard Processing', 'Express', nan], dtype=object)